# Introducing RadioItems Component

In [1]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html, dcc, Input, Output

## 1. Getting and exploring the data

In [2]:
# Load the dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00616/Tetuan%20City%20power%20consumption.csv')

In [3]:
# Changing DateTime column to datetime
df.DateTime = pd.to_datetime(df.DateTime)

In [4]:
# Renaming some columns
df.rename(columns={'Zone 1 Power Consumption': 'Zone1', 'Zone 2  Power Consumption': 'Zone2', 
                   'Zone 3  Power Consumption': 'Zone3'}, inplace=True)

In [5]:
# Getting one value per day
dfd = df.groupby(pd.Grouper(key='DateTime', freq='1D')).mean().reset_index()

In [6]:
# Changing the format
dfd_melted = pd.melt(dfd, id_vars=['DateTime'], value_vars=['Zone1', 'Zone2', 'Zone3'])
dfd_melted.head()

,DateTime,variable,value
0,2017-01-01,Zone1,28465.232067
1,2017-01-02,Zone1,28869.493671
2,2017-01-03,Zone1,30562.447257
3,2017-01-04,Zone1,30689.831224
4,2017-01-05,Zone1,30802.911393


In [7]:
# Renaming columns
dfd_melted = dfd_melted.rename(columns={'variable':'Zone', 'value': 'Power'})
dfd_melted.head()

,DateTime,Zone,Power
0,2017-01-01,Zone1,28465.232067
1,2017-01-02,Zone1,28869.493671
2,2017-01-03,Zone1,30562.447257
3,2017-01-04,Zone1,30689.831224
4,2017-01-05,Zone1,30802.911393


In [8]:
# Choosing Zone3
filtered_zone = dfd_melted[dfd_melted['Zone'] == 'Zone3']
filtered_zone.head()

,DateTime,Zone,Power
728,2017-01-01,Zone3,17868.795181
729,2017-01-02,Zone3,17820.763053
730,2017-01-03,Zone3,17620.803213
731,2017-01-04,Zone3,17673.694779
732,2017-01-05,Zone3,17664.176707


In [9]:
# Plotting the scatter plot of the filtered_data
line_fig = px.line(filtered_zone, x='DateTime', y='Power', 
                    hover_data={'Power':':.2f'},
                    title=f'Power Consumption in Zone 3')
line_fig.update_yaxes(range=[0, dfd.Zone1.max()+5000])                
line_fig.update_traces(line_color='orangered'),
line_fig.show()

## 2. Building the layout 

In [10]:
# Creating the Dash app
app = JupyterDash()

In [11]:
app.layout = html.Div(children=[
    # Defining the title
    html.H1(id='H1', children='Power Consumption in Tetouan, 2017', 
            style={'textAlign':'center', 'color':'orangered'}),
    # Defining a subtitle        
    html.H3(id='H3', children='(Introducing RadioItems)', 
            style={'textAlign':'center'}),                
    # Defining radioitems
    dcc.RadioItems(id='radio-id', options=['Zone1', 'Zone2', 'Zone3'], 
            value='Zone1'),
    # Defining the graph. Notice we use the id 'line-gr'
    dcc.Graph(id='line-gr')
])

## 3. Adding interactivity with callback functions

This dashboard will have one `Input`, the `RadioItems` value, and one `Output`, a graph.

In [12]:
# Set up the callback function
@app.callback(
    Output(component_id='line-gr',  component_property='figure'),
    Input (component_id='radio-id', component_property='value')
)
def update_graph(selected_zone):
    filtered_zone = dfd_melted[dfd_melted['Zone'] == selected_zone]
    line_fig = px.line(filtered_zone, x='DateTime', y='Power', 
                    hover_data={'Power':':.2f'},
                    title=f'Power Consumption in {selected_zone}')
    line_fig.update_yaxes(range=[0, dfd.Zone1.max()+5000])                
    line_fig.update_traces(line_color='orangered'),
    return line_fig

## 4. Running the dashboard

In [13]:
# Run app and display result inline in the notebook
app.run_server(mode='external', port=8087)

Dash app running on http://127.0.0.1:8087/


The dashboard looks like this.

<img src="radioDash.gif" align="center">

## References

- https://dash.plotly.com/dash-core-components/radioitems
- Salam, A., & El Hibaoui, A. (2018) "Comparison of Machine Learning Algorithms for the Power Consumption Prediction: -Case Study of Tetouan city". In 2018 6th International Renewable and Sustainable Energy Conference (IRSEC) (pp. 1-5)